In [40]:
import json 
import re
import pandas as pd 
import chardet
import numpy as np
import os
from pprint import pprint
#from google.cloud import storage
#from google.cloud import language_v1
from dateutil import parser
from datetime import datetime
import matplotlib as plt
import yfinance as yf
from sklearn.metrics import mean_squared_error


In [41]:
## functions to keep code easily modular. 

# loads the intial J_Son file 
def get_file(File):
    enc=chardet.detect(open(File,'rb').read())['encoding']
    with open(File,'r', encoding = enc) as f:
        json_file=json.load(f)
        f.close()
    return json_file

# this formats the json files data as a data frame it requites the users name. 
def get_data_frame(json_file, name): # re-write this to save as data frame and 
    texts=[]
    like_count=[]
    quote_count=[]
    reply_count=[]
    retweet_count=[]
    created_at=[]
    id=[]
    for i in range(len(json_file["data"])):
        texts.append(json_file["data"][i]["text"].lower())
        created_at.append(json_file["data"][i]["created_at"])
        retweet_count.append(json_file["data"][i]['public_metrics']['retweet_count'])
        reply_count.append(json_file["data"][i]['public_metrics']['reply_count'])
        like_count.append(json_file["data"][i]['public_metrics']['like_count'])
        quote_count.append(json_file["data"][i]['public_metrics']['quote_count'])
        id.append(json_file["data"][i]['id'])
    data_frame=pd.DataFrame()
    data_frame["User Name"]=[name]*len(texts)
    data_frame["orignal texts"]=texts
    data_frame["created at"]= created_at
    data_frame["retweet count"]=retweet_count
    data_frame["reply count"]=reply_count
    data_frame["like count"]=like_count
    data_frame["quote count"]=quote_count
    data_frame["id"]=id
    return data_frame

def data_for_period(symbol):
    # takes a stock symbol name as input and outputs and saves csv over the period of analysis for this paper.
    ticker_object=yf.Ticker(symbol)
    historical = ticker_object.history(start="2021-08-01", end="2021-12-01", interval="1d")
    historical.to_csv(symbol+".csv")


def standardized_sum(df, col_names):
    #takes data frame and a list of colomn names sums and standrdizes those coloumsn 
    df["temp"]=df[col_names].sum(axis=1) 
    df["Sum of "+str(col_names)+"Standardized"]=(df["temp"]-np.mean(df["temp"]))/np.std(df["temp"])
    df=df.drop('temp',axis=1)
    return df

# this takes a data frame as well as a pattern and only finds the tweets mathcing that pattenr
def scrape(pattern, data_frame):
    clean_df=data_frame
    for i in range(len(data_frame["orignal texts"])):
        a=re.search(pattern,data_frame["orignal texts"][i])
       # print(a)
        if(a==None):
            data_frame=data_frame.drop(i,axis=0)
            clean_df=clean_text(data_frame)
    return clean_df

# this will make a pattern with a list of our terms outside of the snp 500 from a csv
def get_terms():
    raw_terms=pd.read_excel(r"C:\Users\buzga\Desktop\Coding Stuff\Twitter Reaserch\Data\Pattern\TermsExcel.xlsx")
    pattern_2=""
    for i in range(len(raw_terms["terms"])-1):
        pattern_2=pattern_2+"^"+raw_terms["terms"][i].lower()+"\W|\W"+raw_terms["terms"][i].lower()+"\W|"
    pattern_2=pattern_2+"^"+raw_terms["terms"][i+1].lower()+"\W|\W"+raw_terms["terms"][i+1].lower()+"\W"
    return pattern_2
# this creates the pattern for the scrape method 
def get_pattern(path):
    # for me it is "C:\Users\buzga\Desktop\Coding Stuff\Twitter Reaserch\constituents_csv.csv"
    # either way here is a link to the download: https://datahub.io/core/s-and-p-500-companies#data
    companies=pd.read_csv(path)
    companies=companies.drop("Sector", axis=1)
    # this list is in alphabetical order starting with the company 3m so we skip the first as it would make s add another case. 
    pattern=" "
    for i in range(len(companies["Name"])-1):
        pattern=pattern+"^"+companies["Name"][i].lower()+ "\W|\W"+companies["Name"][i].lower()+"\W|"
        pattern=pattern+"^"+companies["Symbol"][i].lower()+ "\W|\W"+companies["Symbol"][i].lower()+"\W|"
    pattern=pattern+"^"+companies["Name"][len(companies["Name"])-1].lower()+ "\W|\W"+companies["Name"][len(companies["Name"])-1].lower()+"\W|"
    pattern=pattern+"^"+companies["Symbol"][len(companies["Name"])-1].lower()+ "\W|\W"+companies["Symbol"][len(companies["Name"])-1].lower()+"\W"
    return pattern

# this adds a specfic list of words to the SnP500 pattern. 
def new_pattern(path, words):
    pattern=get_pattern(path)
    for i in range(len(words)-1):
        pattern=pattern+"^"+words[i]+"\W|\W"+words[i]+"\W|"
    pattern=pattern+"^"+words[len(words)-1]+"\W|\W"+words[len(words)-1]+"\W|"


# this will import a lexicon 

def get_lexicon(lexicon_path):
    names=["token", "polarity", "standerd deviation", "raw scores"]
    lexicon=pd.read_csv(lexicon_path, sep="\t" , names=(names))
    lexicon=lexicon.set_index("token")
    lexicon.drop(['standerd deviation',"raw scores"], axis=1, inplace=True)
    return lexicon


# this is just a formating method 
def clean_text(data_frame):
    PuncuationPattern = r'[^\w\s]|[â|_]'
    WhiteSpacePattern=r'\s{2,}'
    no_punc=[]
    for text in data_frame["orignal texts"]:
        f=re.sub(PuncuationPattern, " ", text)
        g=re.sub(WhiteSpacePattern, "", f)
        no_punc.append(f) 
    if(len(data_frame)!=0):
        data_frame['cleaned texts'] = no_punc
    else:
        data_frame['cleaned texts'] = ""
    return data_frame

# this will add a coloumn to teh previous data frame called polarity represetning sentminet 
def Lexicon_analyze_sentiment(data_frame,lexicon):
    if("cleaned texts" in data_frame.columns and len(data_frame)!=0 ):
        data_frame=data_frame.set_index("id")
        cleaned_text_split = data_frame["cleaned texts"].str.split(expand = True)
        numbered_columns = cleaned_text_split.columns.values
        cleaned_text_split.reset_index(inplace = True)
        tidy_format = pd.melt(cleaned_text_split, id_vars=['id'], value_vars=numbered_columns)
        tidy_fomat_1=tidy_format.rename(columns={"variable": "num", "value":"word"})
        tidy_fromat_na=tidy_fomat_1.dropna()
        tidy_fromat_sorted=tidy_fromat_na.sort_values(by=['id','num'])
        tidy_format=tidy_fromat_sorted.set_index("id")
        tidy_format_sent_merged= tidy_format.join(lexicon, on="word")
        tidy_format_sent_merged["polarity"]=tidy_format_sent_merged["polarity"].fillna(0)
        data_frame["VADER Sentiment"]=tidy_format_sent_merged.groupby(tidy_format_sent_merged.index).polarity.agg("sum")
    return data_frame

# this will analyze a single stirng usinng google sentment analysis and return its magnitude adn score 
def google_analyze_sentiment_string(text):
    os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"C:\Users\buzga\Desktop\Coding Stuff\Twitter Reaserch\Data\Google api\digital-arcade-333701-1e9b79d00038.json"
    client = language_v1.LanguageServiceClient()
    type_ = language_v1.Document.Type.PLAIN_TEXT
    language = "en"
    document = {"content": text, "type_": type_, "language": language}
    encoding_type = language_v1.EncodingType.UTF8
    response = client.analyze_sentiment(request = {'document': document, 'encoding_type': encoding_type})
    return response.document_sentiment.magnitude,response.document_sentiment.score

# this will analyze a list of strings using google. 
def google_analyze_sentiment(DataFrame):
    sentiment=[]
    magnitude=[]
    if("cleaned texts" in DataFrame.columns):
        for text in DataFrame["cleaned texts"]:
            magnitude.append(google_analyze_sentiment_string(text)[0])
            sentiment.append(google_analyze_sentiment_string(text)[1])
        DataFrame["Google Sentiment"]=sentiment
        DataFrame["Google Magnitude"]=magnitude
    return DataFrame
# assuming our file system is set up a certain way gets us the name and path for quick cast 
def macros(root_path):
    paths=[]
    names=[]
    for files in os.listdir(root_path): 
        for file in os.listdir(root_path+"\\"+files):   
            names.append(files)
            paths.append(root_path+"\\"+files+"\\"+file)
    return paths,names


def standardize(data, bad_cols):
    # data either in the form of a path to a data frame or data frame. 
    # as well as a list of non-numeric cols. 
    # returns a data frame with all cols standardized. 
    data=pd.read_csv(data)
    if(type(data)==str):
        data=pd.read_csv(data)
    col_names=list(data)
    for i in bad_cols:
        col_names.remove(i)
    for i in col_names:
        data[i+" Standardized"]=(data[i]-np.mean(data[i]))/np.std(data[i])
    return data
def single_day(df,date):
    #takes data frame with date col and date, and returns the specfied day
    return df.loc[df['Date'] == date]

def date_in_range(df,start,stop):
    # takes a data frame with a date coloum and finds index's between the two values inclusive
    # where begin and stop are strings. 
    begin=df.index[df['Date'] == start].tolist()
    end=df.index[df['Date'] == stop].tolist()
    return df[begin[0]:end[0]+1]

            
# this will save our data to the computer so that we have it. 
def save_to_csv(data_frame,destination_path):
    data_frame.to_csv(destination_path, mode='a', header=False)


## this will sort the data frame by date. 
def sort_by_date(data):
    temp=[]
    data["created at"][1][0:10]
    for i in data["created at"]:
        temp.append(datetime.strptime(i[0:10],'%Y-%m-%d'))
    data["date"]=temp
    data=data.sort_values(by=["date"])
    #data=data.set_index("date")
    return data 
#this will sort by year month date, and then sum over the sentiment coloumns 
def sumzarize(data):
    df=sort_by_date(data)
    df=df.drop(['id', 'User Name', 'orignal texts', 'created at', 'retweet count',
         'reply count', 'like count', 'quote count', 'cleaned texts'], axis=1)
    df=df.groupby(pd.Grouper(key='date', axis=0, 
                    freq='D')).sum()
    return df

def sum_row(path,bad_cols):
    ## adds a row called sum that is the sum of all other numeric rows. 
    ## path string, path to data fraem 
    ## bad_cols list of non-numeric col names 
    data=pd.read_csv(path)
    col_names=list(data)
    for i in bad_cols:
        col_names.remove(i)
    data["Sum"] = data[col_names].sum(axis=1) 
    return data


def merge_on_date(left,right):
    # does an inner join on the date col for two data frames. 
    return left.merge(right,on="Date", how="inner")

def Summarized_CSV(data):
    ## this takes in data, and saves a csv with summarized sentiment by day. 
    sumzarize(data).to_csv(r'C:\Users\buzga\Desktop\Coding Stuff\Twitter Reaserch\Data\Destination\Sumarized_sentiment.csv')


# this will all the methods that are nesscary to analyze the sentiment in a single json file
def quick_cast(path,name):
    pattern_path=r"C:\Users\buzga\Desktop\Coding Stuff\Twitter Reaserch\Data\Pattern\SnP500.csv"
    lexicon_path=r"C:\Users\buzga\Desktop\Coding Stuff\Twitter Reaserch\Data\vaderSentiment-master\vaderSentiment-master\vaderSentiment\vader_lexicon.txt"
    destination_path=r"C:\Users\buzga\Desktop\Coding Stuff\Twitter Reaserch\Data\Destination\aggregate_tweets.csv"
    data=get_data_frame(get_file(path), name)
    pattern=get_pattern(pattern_path)+"|"+get_terms()
    data=scrape(pattern,data)
    lexicon=get_lexicon(lexicon_path) 
    data=google_analyze_sentiment(data)
    data=Lexicon_analyze_sentiment(data, lexicon)
    save_to_csv(data, destination_path)

# just an over overloaded quick cast to avoid needed reperative function calls.
def overload_quick_cast(path,name,lexicon,pattern,destination_path ):
    data=get_data_frame(get_file(path), name)
    data=scrape(pattern,data)
    data=google_analyze_sentiment(data)
    data=Lexicon_analyze_sentiment(data, lexicon)
    save_to_csv(data, destination_path)


# this takes a list of files and a lists of names and just runs quick cast mulitple times.
def multi_cast(paths, names):
    pattern_path=r"C:\Users\buzga\Desktop\Coding Stuff\Twitter Reaserch\Data\Pattern\SnP500.csv"
    lexicon_path=r"C:\Users\buzga\Desktop\Coding Stuff\Twitter Reaserch\Data\vaderSentiment-master\vaderSentiment-master\vaderSentiment\vader_lexicon.txt"
    destination_path=r"C:\Users\buzga\Desktop\Coding Stuff\Twitter Reaserch\Data\Destination\aggregate_tweets.csv"
    pattern=get_pattern(pattern_path)+"|"+get_terms()
    lexicon=get_lexicon(lexicon_path) 
    for i in range(len(paths)):
        #print(i) # this is just a print traceback in case something is not running correctly. 
        overload_quick_cast(paths[i],names[i],lexicon,pattern,destination_path)
# runs multi path given just a root path. 
def mega_cast(path):
    paths, names= macros(path)
    multi_cast(paths, names)

def quick_read(path,name):
    col_list = ["Date", "Close"]
    df=pd.read_csv(path, usecols=col_list)
    return df.rename(columns={'Close': name+' Close'})

### some new helper functins 
def Stanadrd_graph(data,y,y_name, title):
    ## makes standard line graph 
    ## df is a data frame that must have a date col 
    ## y is new line compared to fundementals 
    ## y_name is the name of new line 
    ## title is tile of graph
    #'Final all Sentiment sum plus Final Standardized sum'
    import matplotlib.pyplot as plt
    X=data["Date"]
    Y1=data['Standardized Macro Indicators']
    Y2=data[y]
    Y3=data['Standardized Sum']
    fig, ax = plt.subplots()
    ax.set_xticks([0,len(data["Date"])/2, len(data["Date"])])
    plt.plot(X, Y1, color="blue", label = "Observed Standardized Movements")
    plt.plot(X, Y2, color="red", label = y_name)
    plt.xlabel("Date")
    plt.ylabel("Deviations from mean")
    plt.title(title)
    plt.legend()
    plt.show()
    mse = mean_squared_error(Y1, Y2)
    print(mse) 
    mse_precent=(mse-mean_squared_error(Y1, Y3))/mean_squared_error(Y1, Y3)
    print(mse_precent*100)
 

def implicit_standardized_sum(df, col_names, new_name):
    #takes data frame and a list of colomn names sums and standrdizes those coloumsn 
    df["temp"]=df[col_names].sum(axis=1) 
    df[new_name]=(df["temp"]-np.mean(df["temp"]))/np.std(df["temp"])
    df=df.drop('temp',axis=1)
    return df
def first_standard_graph(data, period):
    import matplotlib.pyplot as plt
    X=data["Date"]
    Y1=data['Standardized Macro Indicators']
    #Y2=[np.mean(data['Fundementals Final Standrdized Sum'])]*len(data['Fundementals Final Standrdized Sum'])
    fig, ax = plt.subplots()
    ax.set_xticks([0,22, 42,63])
    plt.plot(X, Y1, color="blue", label = "Observed Standardized Movements in Index")
    #plt.plot(X, Y2, color="red", label = "mean")
    plt.xlabel("Date")
    plt.ylabel("Deviations from mean")
    plt.title("Overall Movement in Macroeconomic Indicators Over Period "+period)
    plt.legend()
    plt.show()
#mse = mean_squared_error(Y1, Y2)
#print(mse)
def period_analysis(data_prediod_1, period, start, end):
    
    data_prediod_1=date_in_range(data_prediod_1,start,end)
    data_prediod_1= implicit_standardized_sum(data_prediod_1, ['Standardized Google Sentiment','Standardized Sum' ], "d1")
    data_prediod_1= implicit_standardized_sum(data_prediod_1, ['Standardized Google Magnitude','Standardized Sum' ], "d2")
    data_prediod_1= implicit_standardized_sum(data_prediod_1, ['Standardized VADER Sentiment','Standardized Sum' ], "d3")
    data_prediod_1= implicit_standardized_sum(data_prediod_1, ['Standardized Google Sentiment','Standardized Google Magnitude' ], "t1")
    data_prediod_1= implicit_standardized_sum(data_prediod_1, ['t1','Standardized Sum' ], "d4")
    data_prediod_1= implicit_standardized_sum(data_prediod_1, ['Standardized Google Sentiment','Standardized VADER Sentiment' ], "t2")
    data_prediod_1= implicit_standardized_sum(data_prediod_1, ['t2','Standardized Sum' ], "d5")
    data_prediod_1= implicit_standardized_sum(data_prediod_1, ['Standardized Google Magnitude','Standardized VADER Sentiment' ], "t3")
    data_prediod_1= implicit_standardized_sum(data_prediod_1, ['t3','Standardized Sum' ], "d6")
    data_prediod_1= implicit_standardized_sum(data_prediod_1, ['Standardized Google Sentiment','Standardized Google Magnitude','Standardized VADER Sentiment' ], "t4")
    data_prediod_1= implicit_standardized_sum(data_prediod_1, ['t4','Standardized Sum' ], "d7")
    data_prediod_1= implicit_standardized_sum(data_prediod_1, ['Standardized Fundementals Sum','Standardized Psychology Sum' ], "d8")

    first_standard_graph(data_prediod_1, period)
    # ubtutak bkiinverg graogs 

    Stanadrd_graph(data_prediod_1,"Standardized Fundementals Sum","REH Fundamentals", "REH Fundamentals Compared to Index Movement Period "+period)
    Stanadrd_graph(data_prediod_1,"d8","Behavioral Psychology", "Behavioral Psychology Compared to Index Movement Period "+period)
    Stanadrd_graph(data_prediod_1,"Standardized Sum","KU Model", "Knightian Uncertainty Compared to Index Movement Period "+period)
    ## inittal tiwtter grahs 
    Stanadrd_graph(data_prediod_1,"Standardized Google Sentiment","Google Sentiment", "Google Sentiment Compared to Index Movement Period "+period)
    Stanadrd_graph(data_prediod_1,"Standardized Google Magnitude","Google Magnitude", "Google Magnitude Compared to Index Movement Period "+period)
    Stanadrd_graph(data_prediod_1,"Standardized VADER Sentiment","VADER Sentiment", "VADER Sentiment Compared to Index Movement Period "+period)

    ### overall 
    Stanadrd_graph(data_prediod_1,"d1","Bloomberg + Google Sentiment", "Google Sentiment and Bloomberg Compared to Index Movement Period "+period)
    Stanadrd_graph(data_prediod_1,"d2","Bloomberg + Google Magnitude", "Google Magnitude and Bloomberg Compared to Index Movement Period "+period)
    Stanadrd_graph(data_prediod_1,"d3","Bloomberg + VADER Sentiment", "Google VADER Sentiment and Bloomberg Compared to Index Movement Period "+period)
    Stanadrd_graph(data_prediod_1,"d4","Google Sentiment + Magnitude + Bloomberg", "Google Sentiment and Magnitude Compared to Index Movement Period "+period)
    Stanadrd_graph(data_prediod_1,"d5","Google Sentiment + VADER + Bloomberg" , "Google Sentiment and VADER Compared to Index Movement Period "+period)
    Stanadrd_graph(data_prediod_1,"d6","Google Magnitude + VADER + Bloomberg", "Google Magnitude and VADER Compared to Index Movement Period "+period)
    Stanadrd_graph(data_prediod_1,"d7","Whole Twitter + Bloomberg", "Whole Twitter Sentiment and Bloomberg Compared to Index Movement Period "+period)
